## `sql_connection` for SQL Queries 


This notebook demonstrates how to establish a connection to an existing database using the OOP-based `sql_connection` module. 
The goal is to perform safe, read-only queries against a real database without creating or modifying tables.

**Sample database:** `toys_and_models.sqlite` (located in `examples/`)

You can override the default path by defining the environment variable `SQL_CONN_EXAMPLE_DB`.

---

### Requirements

Follow these steps to set up your environment and run this example successfully:

1. **Clone the repository and create a virtual environment**
    ```bash
    git clone https://github.com/aalopez76/SQL-Connection-Module.git
    cd SQL-Connection-Module

    python -m venv venv
    # On Windows
    venv\Scripts\activate
    # On macOS/Linux
    source venv/bin/activate
    ```
 
2. **Install the package in editable mode**

    ```bash
    pip install -e .
    ```

3. **Install pandas (optional, for DataFrame display)**

    ```bash
    pip install pandas
    ```

4. **Verify installation**

    ```python
    python -c "from sql_connection import get_connector; print('✅ Import OK')"
    ```


---
Below is a complete working example demonstrating database connection, table listing, and read-only querying using SQLite.

### Importing the Module

In [19]:
from sql_connection import get_connector
import os
try:
    import pandas as pd
except ImportError:
    pd = None

### Connecting to SQLite

In [20]:
# Define database path (can be overridden by environment variable)
db_path = os.environ.get("SQL_CONN_EXAMPLE_DB", "toys_and_models.sqlite")
db_path = os.path.abspath(db_path)

if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database not found: {db_path}")

# Create connector and open connection
conn = get_connector("sqlite", path=db_path)
conn.connect()
print("✅ Connected to:", conn.dsn_summary())


✅ Connected to: sqlite:///c:\Users\AALP\SQL-Connection-Module\examples\toys_and_models.sqlite


### Listing Tables

Retrieve all available tables from the SQLite database.

In [21]:
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"

if pd:
    display(conn.read_sql(query_tables))
else:
    rows = conn.query(query_tables)
    print("Tables:", rows)


,name
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


### Basic Queries
Retrieve the first 10 customers

In [22]:
query_customers = "SELECT customerNumber, customerName, country FROM customers LIMIT 10;"

if pd:
    display(conn.read_sql(query_customers))
else:
    rows = conn.query(query_customers)
    for row in rows:
        print(row)


,customerNumber,customerName,country
0,103,Atelier graphique,France
1,112,Signal Gift Stores,USA
2,114,"Australian Collectors, Co.",Australia
3,119,La Rochelle Gifts,France
4,121,Baane Mini Imports,Norway
5,124,Mini Gifts Distributors Ltd.,USA
6,125,Havel & Zbyszek Co,Poland
7,128,"Blauer See Auto, Co.",Germany
8,129,Mini Wheels Co.,USA
9,131,Land of Toys Inc.,USA


### Analytical Query: Top 5 Countries by Customer Count

This query aggregates the number of customers per country and returns the top 5 results.

In [23]:
query_top_countries = """
SELECT country, COUNT(*) AS num_customers
FROM customers
GROUP BY country
ORDER BY num_customers DESC
LIMIT 5;
"""

if pd:
    display(conn.read_sql(query_top_countries))
else:
    rows = conn.query(query_top_countries)
    print("Top countries:", rows)


,country,num_customers
0,USA,36
1,Germany,13
2,France,12
3,Spain,7
4,UK,5


### Parameterized Query (Best Practice)

Use parameterized SQL queries to prevent SQL injection and improve code safety.

In [24]:
country = "USA"
sql_param = """
SELECT customerNumber, customerName, country
FROM customers
WHERE country = :country
LIMIT 5;
"""

params = {"country": country}

if pd:
    display(conn.read_sql(sql_param, params=params))
else:
    rows = conn.query(sql_param, params=params)
    for r in rows:
        print(r)


,customerNumber,customerName,country
0,112,Signal Gift Stores,USA
1,124,Mini Gifts Distributors Ltd.,USA
2,129,Mini Wheels Co.,USA
3,131,Land of Toys Inc.,USA
4,151,Muscle Machine Inc,USA


### Closing the Connection

Always close connections to ensure resources are released properly.

In [25]:
conn.close()
print("Connection closed successfully.")


Connection closed successfully.


### Conclusions

* The `sql_connection` module provides a unified, object-oriented API for interacting with multiple SQL engines (SQLite, PostgreSQL, MySQL, etc.).
* It allows developers and analysts to query heterogeneous data sources seamlessly without modifying connection logic for each engine.
* This example demonstrates secure query execution, context-managed connections, and optional pandas DataFrame integration.
* Ideal for ETL pipelines, analytical workflows, and reproducible data exploration in production environments.